# Agentic Health Triage for Rural Areas

This notebook demonstrates a minimal agentic workflow for basic health
triage designed for low-connectivity environments.


## Objective
Provide basic health orientation and urgency alerts using an offline-capable
agentic workflow built with open models.


# Core imports
import json
from typing import Dict


In [1]:
# Core imports
import json
from typing import Dict


In [2]:
def interpret_symptoms(user_input: str) -> Dict:
    return {
        "interpreted_symptoms": user_input.lower(),
        "confidence": "basic"
    }


In [3]:
def classify_risk(symptoms: Dict) -> str:
    text = symptoms["interpreted_symptoms"]
    if "dor no peito" in text or "falta de ar" in text:
        return "high"
    elif "febre" in text:
        return "medium"
    return "low"


In [4]:
def provide_guidance(risk_level: str) -> str:
    if risk_level == "high":
        return "Procure atendimento médico imediato."
    elif risk_level == "medium":
        return "Observe os sintomas e procure um profissional se persistirem."
    return "Repouso e hidratação podem ajudar."


In [5]:
def agentic_triage_pipeline(user_input: str) -> Dict:
    symptoms = interpret_symptoms(user_input)
    risk = classify_risk(symptoms)
    guidance = provide_guidance(risk)
    
    return {
        "risk_level": risk,
        "guidance": guidance
    }


In [6]:
user_input = "Estou com febre e dor no corpo"
result = agentic_triage_pipeline(user_input)
result


{'risk_level': 'medium',
 'guidance': 'Observe os sintomas e procure um profissional se persistirem.'}